In [1]:
import numpy as np
import matplotlib.pyplot as plt


In [2]:
from qreservoirpy import QReservoir, Layers
res = QReservoir(qubits=4, layers=[
    Layers.H(),
    Layers.Measurement(measure_qubits=[0, 1]),
    Layers.Reset(),
    Layers.Measurement(measure_qubits=[1, 2])
])
fig = res.circuit.draw('mpl')
fig.savefig("Images/simple.jpg", bbox_inches='tight')

In [3]:
from qreservoirpy import QReservoir, Layers
def build_method(circuit, timestep):
    circuit.barrier()
    circuit.measure(circuit.qubits)
    return circuit

res = QReservoir(qubits=4, layers=[
    Layers.H(),
    Layers.Timeseries(build_method=build_method)
])
res.run([0, 1, 2])
fig = res.circuit.draw('mpl')
fig.savefig("Images/simple_timeseries.jpg", bbox_inches='tight')

In [4]:
from qreservoirpy import QReservoir, Layers
from qiskit.quantum_info import random_unitary

def build_method(circuit, timestep, operators, encoder):
    circuit.unitary(operators[timestep], circuit.qubits)
    circuit.measure([0, 1])
    circuit.initialize(encoder[timestep], [0, 1])
    return circuit


timeseries = [0, 1, 2]
res = QReservoir(qubits=4, layers=[
    Layers.H(),
    Layers.Timeseries(build_method=build_method,
        operators = {key : random_unitary(2**4) for key in timeseries}, 
        encoder = {
            0: '00',
            1: '01',
            2: '10',
            3: '11'
        })
])
res.run(timeseries)
fig = res.circuit.draw('mpl')
fig.savefig("Images/adding_parameters.jpg", bbox_inches='tight')

In [5]:
from qreservoirpy import QReservoir, Layers
from qiskit.quantum_info import random_unitary

def build_method(circuit, timestep, operator):
    circuit.unitary(operator, circuit.qubits)
    return circuit


timeseries = [0, 1, 2]
res = QReservoir(qubits=4, layers=[
    Layers.H(),
    Layers.Timeseries(
        build_method=build_method, 
        incrementally=True,
        operator = random_unitary(2**4)
        ),
    Layers.Measurement(range(4))
])
res.run(timeseries)
fig = res.circuit.draw('mpl')
fig.savefig("Images/incrementally.jpg", bbox_inches='tight')

In [8]:
from qreservoirpy import QReservoir, Layers
from qiskit.quantum_info import random_unitary

def analyze_fcn(sim_result):
    return sim_result[-4:]

def build_method(circuit, timestep, operator):
    circuit.measure(0)
    circuit.unitary(operator, circuit.qubits)
    return circuit


timeseries = [0, 1, 2]
res = QReservoir(qubits=4, layers=[
    Layers.H(),
    Layers.Timeseries(
        build_method=build_method, 
        incrementally=True,
        operator = random_unitary(2**4),
        ),
    Layers.Measurement(range(4))
], analyze_function=analyze_fcn)
res.run(timeseries)
fig = res.circuit.draw('mpl')
fig.savefig("Images/nonlinear_incrementally.jpg", bbox_inches='tight')